##Notebook overview:
- In this notebook we load pictures which should be predicted and move those pictures into the determined folder based on the prediction made by the model.
- First we have to install TFOD API (as always)
- Afterwards we create a list of image_paths 
- Then we load the model
- Finally we make predictions and move the pictures into a directory based on the prediction

In [ ]:
#connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Cloning TFOD 2.0 Github** needed for inferencing

In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 69183, done.
remote: Total 69183 (delta 0), reused 0 (delta 0), pack-reused 69183
Receiving objects: 100% (69183/69183), 577.29 MiB | 35.50 MiB/s, done.
Resolving deltas: 100% (48740/48740), done.


In [ ]:
cd /content/models/research

/content/models/research


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 27.92 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [ ]:
cd cocoapi/PythonAPI

/content/models/research/cocoapi/PythonAPI


In [ ]:
!make

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/models/research/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/n

### Install the Object Detection API

In [ ]:
cd /content/models/research

/content/models/research


In [ ]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.1 MB 11.9 MB/s 
     |████████████████████████████████| 352 kB 60.3 MB/s 
     |████████████████████████████████| 2.2 MB 54.8 MB/s 
     |████████████████████████████████| 23.4 MB 1.3 MB/s 
     |████████████████████████████████| 234 kB 72.0 MB/s 
     |████████████████████████████████| 90 kB 10.0 MB/s 
     |████████████████████████████████| 636 kB 63.1 MB/s 
     |████████████████████████████████| 1.2 MB 49.7 MB/s 
     |████████████████████████████████| 99 kB 10.8 MB/s 
     |████████████████████████████████| 47.7 MB

In [ ]:
# From within TensorFlow/models/research/
#test if tensorflow downloaded successfully
!python object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-02-18 12:03:35.408093: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0218 12:03:35.977623 139655168583552 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 3.97s
I0218 12:03:36.479960 139655168583552 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 3.97s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.85s
I0218 12:03:37.331681 139655168583552 test

## Create a list of image_paths

In [ ]:
import os

In [ ]:
#create a list of pcitures filepaths
image_dir= '/content/drive/MyDrive/training_demo/images/Tchibo_images' 
files = os.listdir(image_dir)
#print(files)
image_list = []
for file in files:
  if file.endswith("jpg"):
    image_list.append(image_dir +"/" + file)
print(len(image_list))

image_list=image_list[18000:]
print(len(image_list))

23380
6880


##Load the model

In [ ]:
import tensorflow as tf

In [ ]:
# load the model

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

path_to_saved_model = "/content/drive/MyDrive/training_demo/new_exported_models/Exported_centernet_512/saved_model"

print('Loading model...', end='')
start_time = time.time()

# load saved model 
detect_fn = tf.saved_model.load(path_to_saved_model)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...

Done! Took 92.06733775138855 seconds


## Predict pictures and move them into directory based on the prediction

If there are problems with cv2 ("registerMatetype" or something like that) run the cell below and restart runtime

In [ ]:
!pip uninstall opencv-python
!pip install opencv-python==4.5.2.52
!pip uninstall opencv-contrib-python
!pip install opencv-contrib-python==4.5.2.52
!pip uninstall opencv-python-headless
!pip install opencv-python-headless==4.5.2.52

In [ ]:

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse

# provide path to label_map
path_to_labels = '/content/drive/MyDrive/training_demo/annotations/label_map.pbtxt'

# load label map
category_index = label_map_util.create_category_index_from_labelmap(path_to_labels,
                                                                    use_display_name=True)
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import shutil

counter=0 #counts how many pics were detected and moved to folder
i=0
for image_path in image_list: #assign index with enumerate didnt't work
  i +=1
  print(i) 
  image = cv2.imread(image_path)
  #some pictures can not be grabed by cv2.cvtColor. Haven't figured out why ao I handled it with a try/except.
  try:
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  except:
    continue 
  image_expanded = np.expand_dims(image_rgb, axis=0)

  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis, ...]

  # input_tensor = np.expand_dims(image_np, 0)
  detections = detect_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
  detections['num_detections'] = num_detections

  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  image_with_detections = image.copy()
  #if the first detection is a Tchibologo and detection_score is >0.5 move the pic in determined directory
  if detections['detection_classes'][0] == 1 and (detections['detection_scores'][0] < 0.75 and detections["detection_scores"][0] > 0.5):
    source = image_path
    destination = '/content/drive/MyDrive/training_demo/images/tcom_images_labeled/Tchibologo_belwo_75_bigger_50_part2'+"/image_tcom" +str(i) +"below_50_Tchibologo_part2_valscore_"+str(detections['detection_scores'][0])+ ".jpg"
    shutil.move(source,destination)
    counter +=1
    print("Bilder verschoben:" + str(counter))
'''
   #else if the first detection is a Gesicht and detection_score is >0.5 move the pic in determined directory
  elif detections['detection_classes'][0] == 2 and detections['detection_scores'][0] >= 0.5:
    source = image_path
    destination = '/content/drive/MyDrive/Communirty_tchibo_Gesichter'+"/image" +str(i) +"_Gesicht_valscore"+str(detections['detection_scores'][0])+ ".jpg"
    shutil.move(source,destination)
    counter +=1
    print("Bilder verschoben:" + str(counter))
'''

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
217

'\n   #else if the first detection is a Gesicht and detection_score is >0.5 move the pic in determined directory\n  elif detections[\'detection_classes\'][0] == 2 and detections[\'detection_scores\'][0] >= 0.5:\n    source = image_path\n    destination = \'/content/drive/MyDrive/Communirty_tchibo_Gesichter\'+"/image" +str(i) +"_Gesicht_valscore"+str(detections[\'detection_scores\'][0])+ ".jpg"\n    shutil.move(source,destination)\n    counter +=1\n    print("Bilder verschoben:" + str(counter))\n'